In [ ]:
# 必要なライブラリをインポートする
import numpy as np
import pandas as pd
import re

# DataFrame 内の URL を完全に表示するようにする
pd.set_option('display.max_colwidth', None)

In [ ]:
#東京都23区の物件を個別に保存しておいたものを読み込み、結合
data = pd.read_csv('data/suumo_23_wards.csv', encoding='utf-8')
df = pd.concat([data], axis=0, ignore_index=True).reset_index(drop=True)
df[:10]

In [ ]:
# 立地を「最寄駅」と「徒歩〜分」に分割する
splitted1 = df['立地1'].str.split(' 歩', expand=True)
splitted1.columns = ['立地11', '立地12']
splitted2 = df['立地2'].str.split(' 歩', expand=True)
splitted2.columns = ['立地21', '立地22']
splitted3 = df['立地3'].str.split(' 歩', expand=True)
splitted3.columns = ['立地31', '立地32']
# 分割したカラムを結合する
df = pd.concat([df, splitted1, splitted2, splitted3], axis=1)
# 分割前のカラムは分析に使用しないので削除しておく
df.drop(['立地1','立地2','立地3'], axis=1, inplace=True)

# 変な立地情報を削除する
# df.loc[df['立地31'] == '自01/02自由が丘駅-駒大/深沢小学校', ['立地31']] = None

# 数値として扱いたいので、不要な文字列を削除する
df['築年数'] = df['築年数'].str.replace('新築', '0') # 新築は築年数 0 年とする
df['築年数'] = df['築年数'].str.replace(r'築(\d+)年', r'\1')
df['賃料'] = df['賃料'].str.replace('万円', '')
df['管理費'] = df['管理費'].str.replace('円', '')
df['専有面積'] = df['専有面積'].str.replace('m2', '')
df['立地12'] = df['立地12'].str.replace('分', '')
df['立地22'] = df['立地22'].str.replace('分', '')
df['立地32'] = df['立地32'].str.replace('分', '')
df['敷金'] = df['敷金'].str.replace('万円', '')
df['礼金'] = df['礼金'].str.replace('万円', '')

df[:10]

In [ ]:
# "-" を 0 に変換する
df['管理費'] = df['管理費'].replace('-', 0)
df['敷金'] = df['敷金'].replace('-', 0)
df['礼金'] = df['礼金'].replace('-', 0)

# 文字列から数値に変換する
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])
df['立地12'] = pd.to_numeric(df['立地12'])
df['立地22'] = pd.to_numeric(df['立地22'])
df['立地32'] = pd.to_numeric(df['立地32'])

# 単位を「万円」から「円」にするために、管理費以外を 10000 倍する
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000

# 管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']

# 敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['敷金+礼金'] = df['敷金'] + df['礼金']

df[:10]

In [ ]:
# 住所を「東京都」「〜区」「市町村番地」に分割する
splitted6 = df['住所'].str.split('区', expand=True)
splitted6.columns = ['区', '市町村']
splitted6['区'] = splitted6['区'] + '区'
splitted6['区'] = splitted6['区'].str.replace('東京都', '')

# 立地を「路線」「駅」「徒歩〜分」に分割する
splitted7 = df['立地11'].str.split('/', expand=True)
splitted7.columns = ['路線1', '駅1']
splitted7['徒歩1'] = df['立地12']
splitted8 = df['立地21'].str.split('/', expand=True)
splitted8.columns = ['路線2', '駅2']
splitted8['徒歩2'] = df['立地22']
splitted9 = df['立地31'].str.split('/', expand=True)
splitted9.columns = ['路線3', '駅3']
splitted9['徒歩3'] = df['立地32']

# 上記のカラムを結合する
df = pd.concat([df, splitted6, splitted7, splitted8, splitted9], axis=1)
# 不要なカラムを削除する
df.drop(['立地11','立地12','立地21','立地22','立地31','立地32'], axis=1, inplace=True)

# 階を数値化する; 地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'] = splitted10['階1'].str.replace('階', '')
splitted10['階1'] = splitted10['階1'].str.replace('B', '-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])

df = pd.concat([df, splitted10], axis=1)

# 建物高さを数値化する; 地下は無視する
df['建物高さ'] = pd.to_numeric(df['建物高さ'].str.replace(r'(地下\d+地上)?(\d+)階建', r'\2').str.replace('平屋', '1'))

# index を振り直す（これをしないと、後続の処理でエラーが出る）
df = df.reset_index(drop=True)

df[:10]

In [ ]:
# 間取りを「部屋数」「DK有無」「K有無」「L有無」「S有無」に分割する
df['間取りDK'] = 0
df['間取りK'] = 0
df['間取りL'] = 0
df['間取りS'] = 0
df['間取り'] = df['間取り'].str.replace('ワンルーム', '1') # ワンルームを 1 に変換する

for symb in ['DK', 'K', 'L', 'S']:
    df.loc[df['間取り'].str.contains(symb), '間取り' + symb] = 1

df['間取り'] = df['間取り'].str.replace(r'[DKLS]', '')
df['間取り'] = pd.to_numeric(df['間取り'])

df[:10]

In [ ]:
#カラムを入れ替えて、csvファイルとして出力
df = df[['マンション名', '住所', '区', '市町村', '間取り', '間取りDK', '間取りK', '間取りL', '間取りS', '築年数', '建物高さ',
         '階1', '専有面積', '敷金+礼金', '路線1', '駅1', '徒歩1', '路線2', '駅2', '徒歩2', '路線3', '駅3', '徒歩3',
         '賃料', '管理費', '賃料+管理費', '敷金', '礼金', '敷金+礼金', 'Link']]

%time df.to_csv('data/suumo_tokyo.csv', encoding='utf-8')

In [ ]:
df[:10]